# Training logic

In [1]:
import torch
from torch import nn 
from torch import optim
import torch.nn.functional as F

from torchvision import datasets, transforms

##### 디바이스 설정

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

##### 데이터 로더 설정

In [4]:
batch_size = 32

train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('dataset/', train=True, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize(mean=(0.5,), std=(0.5,))
                   ])),
    batch_size=batch_size,
    shuffle=True)

##### 딥러닝 모델 구축

In [5]:
class ResidualBlock(nn.Module):
    def __init__(self, in_channel, out_channel):
        super(ResidualBlock, self).__init__()
        
        self.in_channel, self.out_channel = in_channel, out_channel
        
        self.conv1 = nn.Conv2d(in_channel, out_channel, kernel_size=1, padding=0)
        self.conv2 = nn.Conv2d(out_channel, out_channel, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(out_channel, out_channel, kernel_size=1, padding=0)
        
        if in_channel != out_channel:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channel, out_channel, kernel_size=1, padding=0)
            )
        else:
            self.shortcut = nn.Sequential()
    
    def forward(self, x):
        out = F.relu(self.conv1(x))
        out = F.relu(self.conv2(out))
        out = F.relu(self.conv3(out))
        out = out + self.shortcut(x)
        return out

class ResNet(nn.Module):
    def __init__(self, color='gray'):
        super(ResNet, self).__init__()
        if color == "gray":
            self.conv1 = nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1)
        elif color == "rgb":
            self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1)
            
        self.resblock1 = ResidualBlock(32, 64)
        self.resblock2 = ResidualBlock(64, 64)
        
        self.avgpool = nn.AdaptiveAvgPool2d((1,1))
        self.fc1 = nn.Linear(64, 64)
        self.fc2 = nn.Linear(64, 10)
        
    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2, 2)
        x = self.resblock1(x)
        x = self.resblock2(x)
        x = self.avgpool(x)
        x = torch.flatten(x,1)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        x = F.log_softmax(x, dim=1)
        return x

##### 모델 생성

In [6]:
model = ResNet().to(device)

In [7]:
print(model)

ResNet(
  (conv1): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (resblock1): ResidualBlock(
    (conv1): Conv2d(32, 64, kernel_size=(1, 1), stride=(1, 1))
    (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (conv3): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1))
    (shortcut): Sequential(
      (0): Conv2d(32, 64, kernel_size=(1, 1), stride=(1, 1))
    )
  )
  (resblock2): ResidualBlock(
    (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1))
    (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (conv3): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1))
    (shortcut): Sequential()
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(1, 1))
  (fc1): Linear(in_features=64, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=10, bias=True)
)


### - Learning Rate Scheduler

In [12]:
optimizer = optim.SGD(model.parameters(), lr=0.003)

In [13]:
from torch.optim.lr_scheduler import ReduceLROnPlateau

In [14]:
scheduler = ReduceLROnPlateau(optimizer, mode='min', verbose=True)

- loss를 사용할때는 mode = min
- accuracy는 mode = max
- verbose = True : 기록 출력

In [17]:
def train_loop(dataloader, model, loss_fn, optimizer, scheduler, epoch):
    model.train()
    size = len(dataloader)
    for batch, (x, y) in enumerate(dataloader):
        x, y = x.to(device), y.to(device)
        
        pred = model(x)
        loss = loss_fn(pred, y)
        
        optimizer.zero_grad() # 기울기 초기화
        loss.backward()       # 미분값 계산
        optimizer.step()      # 가중치 업데이트
        
        if batch % 100 == 0: # batch 100번 마다 기록 출력
            loss = loss.item()
            print(f"epoch {epoch+1} : [{batch}/{size}] loss : {loss}")
            
    scheduler.step(loss) # epoch마다 스케쥴작업
    
    return loss.item()

In [18]:
for epoch in range(10):
    epoch_loss = train_loop(train_loader, model, F.nll_loss, optimizer, scheduler, epoch)
    print('                                              ')
    print(f"Epoch:{epoch+1} Total loss:{epoch_loss}" ) # Total이 맞는 표현일까?? 마지막 loss면 Last 아닐까?
    print('------------------------------------------------')
    

epoch 1 : [0/1875] loss : 1.7300424575805664
epoch 1 : [100/1875] loss : 1.7123202085494995
epoch 1 : [200/1875] loss : 1.908084511756897
epoch 1 : [300/1875] loss : 1.741002082824707
epoch 1 : [400/1875] loss : 1.6412979364395142
epoch 1 : [500/1875] loss : 1.3772528171539307
epoch 1 : [600/1875] loss : 1.6604217290878296
epoch 1 : [700/1875] loss : 1.8138378858566284
epoch 1 : [800/1875] loss : 1.714860200881958
epoch 1 : [900/1875] loss : 1.7126504182815552
epoch 1 : [1000/1875] loss : 1.841030240058899
epoch 1 : [1100/1875] loss : 1.5421550273895264
epoch 1 : [1200/1875] loss : 1.491504192352295
epoch 1 : [1300/1875] loss : 1.9426727294921875
epoch 1 : [1400/1875] loss : 1.5088533163070679
epoch 1 : [1500/1875] loss : 1.9766641855239868
epoch 1 : [1600/1875] loss : 1.8407410383224487
epoch 1 : [1700/1875] loss : 1.6930999755859375
epoch 1 : [1800/1875] loss : 1.6835341453552246
                                              
Epoch:1 Total loss:1.625927448272705
---------------------

epoch 9 : [100/1875] loss : 1.2949743270874023
epoch 9 : [200/1875] loss : 1.129211664199829
epoch 9 : [300/1875] loss : 1.4545925855636597
epoch 9 : [400/1875] loss : 1.1834051609039307
epoch 9 : [500/1875] loss : 1.1695116758346558
epoch 9 : [600/1875] loss : 1.4029104709625244
epoch 9 : [700/1875] loss : 1.155512809753418
epoch 9 : [800/1875] loss : 1.4550851583480835
epoch 9 : [900/1875] loss : 1.2941515445709229
epoch 9 : [1000/1875] loss : 1.361046314239502
epoch 9 : [1100/1875] loss : 1.4774689674377441
epoch 9 : [1200/1875] loss : 1.175002098083496
epoch 9 : [1300/1875] loss : 1.0194416046142578
epoch 9 : [1400/1875] loss : 1.5190025568008423
epoch 9 : [1500/1875] loss : 1.461818814277649
epoch 9 : [1600/1875] loss : 1.3104019165039062
epoch 9 : [1700/1875] loss : 1.399703025817871
epoch 9 : [1800/1875] loss : 1.2219456434249878
                                              
Epoch:9 Total loss:1.2137383222579956
----------------------------------------------
epoch 10 : [0/1875]